In [21]:
import pandas as pd
import numpy as np
import re

'''
{'讨价还价':100, '说出目的':101, '确认数额':102, '请求重复':103, '请求等下打来':104, '其它通讯方式':105, '模糊确认':106, '回问身份':107, '还款方式':108, '故意岔开话题':109, '不愿配合':110}
'''
mapping = {1:101,
 2:110,
 3:102,
 4:105,
 5:106, 
 6:108,
 7:107}

In [22]:
df = pd.read_csv('../data/data_other_20180706.csv',encoding='utf-8')
# df = df.dropna(axis = 1)
# df = df.iloc[105:]
df

,序号,提交答卷时间,来自IP,第1题(1),第1题(2),第1题(3),第1题(4),第1题(5),第1题(6),第1题(7),...,第7题(6),第7题(7),第7题(8),第7题(9),第7题(10),第7题(11),第7题(12),第7题(13),第7题(14),第7题(15)
0,2,2018-07-04 11:42:14,125.69.42.16-成都,你是哪位？,你都不知道我是谁还打电话给我？你到底要做什么？,你是哪位？打电话有啥子？,先把你的身份说出来再问我是谁,我认不认识他管你什么事情？,是找他做什么？,我不是，你有啥事情？我帮你找他,...,你哪位？哪里凉快去哪里,鬼大爷晓得你不是债务公司的人啊,你怎么证明我欠你们钱？,为什么要还你钱？你是谁,我是欠钱，但是只欠公司的钱，不欠一个随便打电话来的陌生人的钱,我人都不认识你，怎么会欠你钱,麻烦先让我确认下你的身份再说,NaN,NaN,NaN
1,18,2018-07-06 17:47:52,125.69.41.104-成都,你要干嘛,你想爪子,你谁啊,有话快说,有事说事,干嘛，你想干啥？,你谁啊，你管我，到底想干嘛,...,我来电显示这是：疑似骗子电话,我挺你口音不是成都的哇？哪里人啊,你好久去的你们公司哦？你们那是骗子高利贷公司哦,小妹，你好久去的你们公司哦，我劝你还是赶紧离职哦，别被骗了,美女，你年纪轻轻干啥不好啊，别给高利贷打工了！,你说你是XX公司就是啊，杂证明啊？,我又没问你借钱，你算老几哦？,NaN,NaN,NaN
2,3,2018-07-04 15:19:27,125.69.42.18-成都,你找他什么事,有事吗？,有什么事情,你要干嘛,你要干什么,你打电话有什么事,什么事？,...,who are you?,兄弟，你是谁？,兄弟，你哪位？,你不会是骗子吧？,你是哪个？,你哪位？,你是？？,NaN,NaN,NaN
3,4,2018-07-04 15:52:46,125.69.42.16-成都,什么事？,怎么了？,说事,你要干嘛,快说,快说事，我没时间,你想干什么,...,我靠，我的信息又被卖了,先告诉我你是谁,你先说，我在说,你说你是谁？,我现在知道你是谁？,你怎么知道的？,你怎么知道我的电话？,NaN,NaN,NaN
4,5,2018-07-04 16:00:01,125.69.42.20-成都,你想爪子,他在办事，我可以转告,你找他干什么,你找他事催款吗？,啥子事情？,有什么事？,有话直说,...,你是不会帅哥,你是不是美女,你多大了,你耍朋友没,你有老婆吗？,你住北京？,你小孩几岁？,NaN,NaN,NaN
5,6,2018-07-05 9:42:16,125.69.41.104-成都,有什么事,有事请讲,打电话干什么,干嘛,干煞,赶啥类,作甚个呀,...,我怎么知道你是不是骗子,你是骗子吧,你先告诉我你是谁,你怎么是用私人电话打的,我借了很多家，你是哪家的？,我认识你吗,对不起，我不认识你,NaN,NaN,NaN
6,7,2018-07-05 11:14:14,182.131.10.29-成都,你要做爪子,说，啥子事,你是谁,who are you,干嘛,啊？,有事吗,...,嗯嗯，你谁啊,哼，你别扰我,你号码显示诈骗啦？,你工号？,你哪个公司的？,我怎么知道你不是骗子啦？,呵呵，给我验证一下你是不是骗子,NaN,NaN,NaN
7,8,2018-07-05 16:52:01,125.69.41.107-成都,是的，有事吗,对，干嘛,有什么事说,找我有什么事情吗,神马，有事吗,怎么，有事快说，我在开车,你说是就是，有什么事快说,...,你是不是催债的啊,你哪家公司的,你是做什么的,你是哪里的，我要告你骚扰,刚才电话是不是也是你打的,催收公司是什么，我欠钱的是银行啊,你怎么会有我的联系方式，怎么知道我欠钱了呢,NaN,NaN,NaN
8,9,2018-07-06 10:56:12,125.69.41.112-成都,你要咋子,你到底要咋子,你龟儿子吃饱了所,你瓜娃子所,你是胎神吗,我龟儿脑壳有包吗,你到底想啷个,...,你信不信我弄死你,为啥给你说哦,你长得乖所,你想多了,你真的是想的出来哦,啥子东东,你哪知道我的哦,NaN,NaN,NaN
9,11,2018-07-06 13:56:32,125.69.41.112-成都,你究竟有什么目的,你打电话干啥,干啥,怎么了,你说吧，什么事,什么情况,你想干什么,...,"兄die,我为撒子要给你说，你谁啊",你是骗子吧，我手机显示你是骚扰电话,啥子才告诉你,你嚯哪个,我的信息为啥要透露,你吃多了,二货,NaN,NaN,NaN


In [23]:
result = {}
columns = df.columns.values

for i in range(1,8):
    expression = r'(?u)第{}题'.format(i)
    text = []
    delete = []
    for column in columns:
        if len(re.findall(expression,column)) > 0:
            delete.append(False)
            text.extend(list(df[column].values))
        else:
            delete.append(True)
    label = [mapping[i] for a in range(len(text))]
    columns = columns[delete]
    cur = {mapping[i]:{'split_text':text, 'label':label}}
    result[i] = cur

# Combine

In [24]:
com_dict = {}
for each in result:
    key = list(result[each].keys())[0]
    if com_dict.get(key) is None:
        com_dict[key] = []
    com_dict[key].append(pd.DataFrame(result[each][key]))
result = {}
for each in com_dict:
    result[each] = pd.concat(com_dict[each])

In [25]:
for each in result:
    result[each] = result[each].dropna()
    
other_result = pd.concat([result[101],result[110],result[102],result[105],result[106],result[108],result[107]])

In [26]:
result.keys()

dict_keys([101, 110, 102, 105, 106, 108, 107])

In [27]:
other_result

,split_text,label
0,你是哪位？,101
1,你要干嘛,101
2,你找他什么事,101
3,什么事？,101
4,你想爪子,101
5,有什么事,101
6,你要做爪子,101
7,是的，有事吗,101
8,你要咋子,101
9,你究竟有什么目的,101


# SAVE

In [72]:
# path = '../../MLModel/data/{}/mock_up_data.csv'
# path_save = '../../MLModel/data/{}/mock_up_data1.csv'

In [73]:
# for each in result.keys():
#     cur_path = path.format(each)
#     print(path.format(each))
#     df_old = pd.read_csv(path.format(each), encoding='utf-8')
#     df_new = pd.concat([df_old, result[each]])
#     df_new.to_csv(path_save.format(each),encoding='utf-8',index=False)

# Save In a New Path

In [13]:
path_save = '../../MLModel/data/others/mock_up_others_data0706.csv'
for each in result.keys():
    print(path_save.format(each))
    result[each].to_csv(path_save.format(each),encoding='utf-8',index=False)

../../MLModel/data/others/mock_up_others_data0706.csv
../../MLModel/data/others/mock_up_others_data0706.csv
../../MLModel/data/others/mock_up_others_data0706.csv
../../MLModel/data/others/mock_up_others_data0706.csv
../../MLModel/data/others/mock_up_others_data0706.csv
../../MLModel/data/others/mock_up_others_data0706.csv
../../MLModel/data/others/mock_up_others_data0706.csv
